In [5]:
import os, sys
import glob
import time
from datetime import timedelta, datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
#setting offilne
plotly.offline.init_notebook_mode(connected=True)
import pyarrow.parquet as pq
import json
from gezi import tqdm
import gezi
import qgrid

In [6]:
def gtime(timestamp):
  x = time.localtime(timestamp)
  return f'{x.tm_mon}-{x.tm_mday}'

In [7]:
for i in reversed(range(30)):
  print(i, end='\r')
  dc = gezi.read_parquet(f'../input/train/part_{i + 1}/context.parquet')
  dids = set(dc.did.values)
  vids = set(dc.vid.values)
  dc['id'] = dc.did + '\t' + dc.vid.astype(str)  
  ids = set(dc.id)
  du = gezi.read_parquet(f'../input/train/part_{i + 1}/user.parquet')
  du['watches'] =  du.watch.apply(json.loads)
  now_time = gtime(dc.timestamp.values[0])
  did_vids = []
  for _, row in tqdm(du.iterrows(), total=len(du)):
    did = row['did']
    watches = row['watches']
    for timestamp, vid in watches:
      if gtime(timestamp) == now_time:
        id_ = did + str(vid)
        if vid in vids and id_ not in ids:
          did_vids += [(did, vid, timestamp)]
  dids, vids, timestamps = zip(*did_vids)
  dvs = pd.DataFrame({'did': dids, 'vid': vids, 'timestamp': timestamps})
  dc_vid = dc[['vid', 'prev']].drop_duplicates()
  dvs = dvs.merge(dc_vid, on='vid', how='left')
  dc_did = dc.drop(['label', 'index', 'prev', 'id', 'timestamp', 'vid'], axis=1)
  dc_did = dc_did.drop_duplicates()
  dvs = dvs.merge(dc_did, on='did', how='left')
  dvs['label'] = 1
  dvs['index'] = -1
  dc = dc.drop(['id'], 1)
  dvall = pd.concat([dc, dvs])
  dvall.to_csv(f'../input/train/part_{i + 1}/context.csv', index=False)

100%|██████████| 92295/92295 [00:34<00:00, 2644.21it/s]


KeyboardInterrupt: 

In [ ]:
dc = gezi.read_parquet(f'../input/eval/context.parquet')
dids = set(dc.did.values)
vids = set(dc.vid.values)
dc['id'] = dc.did + '\t' + dc.vid.astype(str)  
ids = set(dc.id)
du = gezi.read_parquet(f'../input/train/eval/user.parquet')
du['watches'] =  du.watch.apply(json.loads)
now_time = gtime(dc.timestamp.values[0])
did_vids = []
for _, row in tqdm(du.iterrows(), total=len(du)):
  did = row['did']
  watches = row['watches']
  for timestamp, vid in watches:
    if gtime(timestamp) == now_time:
      id_ = did + str(vid)
      if vid in vids and id_ not in ids:
        did_vids += [(did, vid, timestamp)]
dids, vids, timestamps = zip(*did_vids)
dvs = pd.DataFrame({'did': dids, 'vid': vids, 'timestamp': timestamps})
dc_vid = dc[['vid', 'prev']].drop_duplicates()
dvs = dvs.merge(dc_vid, on='vid', how='left')
dc_did = dc.drop(['label', 'index', 'prev', 'id', 'timestamp', 'vid'], axis=1)
dc_did = dc_did.drop_duplicates()
dvs = dvs.merge(dc_did, on='did', how='left')
dvs['label'] = 1
dvs['index'] = -1
dc = dc.drop(['id'], 1)
dvall = pd.concat([dc, dvs])
dvall.to_csv(f'../input/train/eval/context.csv', index=False)

In [8]:
dce = gezi.read_parquet(f'../input/eval/context.parquet')
dce.to_csv('../input/eval/context.csv', index=False)